## Load IMDB Data set
Only keep the top 10,000 most frequently occurring words in the data

In [1]:
from keras.datasets import imdb
# Load the data as lists of integers.
max_words = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_words)

/home/ubuntu/anaconda/envs/py36-venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Get the distribution of review lengths

In [2]:
import numpy as np
import matplotlib.pyplot as plt

lens = list(map(len, x_train))
print('max, min, mean: {}'.format((np.max(lens), np.min(lens), np.mean(lens))))
plt.hist(lens)
plt.show()

max, min, mean: (2494, 11, 238.71364)


<Figure size 640x480 with 1 Axes>

## Pad or truncate each review to make the same length

In [3]:
from keras.preprocessing.sequence import pad_sequences

max_review_len = 500
x_train = pad_sequences(x_train, maxlen=max_review_len)
x_test = pad_sequences(x_test, maxlen=max_review_len)

x_train[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [4]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding, Dropout

## Vectorization: Word Embeddings

Embedding Matrix is [10000, 20]

In [5]:
feature_dim = 20

model = Sequential()
model.add(Embedding(max_words, feature_dim, input_length=max_review_len))

model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.7)),
model.add(Dense(1, activation='sigmoid'))

In [6]:
from keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy']
             )

In [7]:
x_val = x_train[:10000]
x_train = x_train[10000:]

y_val = y_train[:10000]
y_train = y_train[10000:]

## Train and Test the model

In [8]:
history = model.fit(x_train, y_train,
                    epochs=2,
                    batch_size=32,
                    validation_data=(x_val, y_val))

Train on 15000 samples, validate on 10000 samples
Epoch 1/2
15000/15000 [==============================] - 3s 226us/step - loss: 0.5490 - acc: 0.6929 - val_loss: 0.3271 - val_acc: 0.8594
Epoch 2/2
15000/15000 [==============================] - 3s 190us/step - loss: 0.2571 - acc: 0.9013 - val_loss: 0.2870 - val_acc: 0.8825


In [9]:
model.evaluate(x_test, y_test)

25000/25000 [==============================] - 1s 55us/step


[0.2927669722890854, 0.87752]

In [10]:
preds = model.predict_classes(x_test[:4])
probs = model.predict_proba(x_test[:4])

print("Predicted Classes: {}".format(preds))
print("Predicted Probabilities:\n {}".format(probs))

Predicted Classes: [[0]
 [1]
 [1]
 [0]]
Predicted Probabilities:
 [[0.05055669]
 [0.9987041 ]
 [0.72994906]
 [0.31417215]]


## Embedding Matrix Weight

In [11]:
weights = model.get_weights() # get model weights
embeddings = weights[0]
print("The Embedding Layer's Shape: {}\n".format(embeddings.shape))
embeddings[:3]

The Embedding Layer's Shape: (10000, 20)



array([[-0.00040193,  0.00052042, -0.00347654, -0.00111846,  0.00516806,
         0.00054633, -0.00222922,  0.00071461,  0.0035489 , -0.00062467,
        -0.00368488,  0.00255503, -0.00199691,  0.00034472, -0.00360599,
         0.0020104 ,  0.00608825,  0.00419473, -0.00108733, -0.00360193],
       [-0.00930036, -0.00112146,  0.0585938 ,  0.02294737, -0.03802411,
         0.00203338,  0.04374479, -0.02325076,  0.00796256,  0.02651064,
         0.06300712, -0.02150084, -0.04345286, -0.01683675,  0.06180406,
        -0.00233442, -0.03593991, -0.07857389, -0.00209842, -0.06425215],
       [ 0.04249636, -0.00852364,  0.00597049,  0.01763395, -0.02088473,
         0.02338243, -0.02212349, -0.00328648,  0.00600215, -0.01482508,
        -0.03491296, -0.04678832, -0.02405351, -0.00277475, -0.00914833,
         0.00908425,  0.04600341,  0.03514171,  0.0234772 , -0.05848712]],
      dtype=float32)

## Transfer Learning

The pre-computed embeddings from 2014 English Wikipedia. It's a 822MB zip file named glove.6B.zip

In [12]:
#! wget http://nlp.stanford.edu/data/glove.6B.zip

#! unzip glove.6B.zip
#! cp glove.6B.100d.txt ./models/

## Load GloVe weight file to a dictionary

In [13]:
import os

glove_dir = './models/'
embeddings_dict = {}

f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_dict[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_dict))

Found 400000 word vectors.


## Build a Embedding Matrix for GloVe weights

In [14]:
word_index = imdb.get_word_index()
feature_dim = 100

embedding_matrix = np.zeros((max_words, feature_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_dict.get(word)
    if i < max_words and (embedding_vector is not None):
        embedding_matrix[i] = embedding_vector


In [15]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, feature_dim, input_length=max_review_len))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 100)          1000000   
_________________________________________________________________
flatten_2 (Flatten)          (None, 50000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1600032   
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 2,600,065
Trainable params: 2,600,065
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [17]:
model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_data=(x_val, y_val))

model.save_weights('./models/pre_trained_glove_model.h5')

Train on 15000 samples, validate on 10000 samples
Epoch 1/20
15000/15000 [==============================] - 4s 237us/step - loss: 0.7420 - acc: 0.4997 - val_loss: 0.6932 - val_acc: 0.4948
Epoch 2/20
15000/15000 [==============================] - 3s 223us/step - loss: 0.7011 - acc: 0.5041 - val_loss: 0.6932 - val_acc: 0.4948
Epoch 3/20
15000/15000 [==============================] - 3s 223us/step - loss: 0.7018 - acc: 0.5039 - val_loss: 0.6931 - val_acc: 0.5053
Epoch 4/20
15000/15000 [==============================] - 3s 222us/step - loss: 0.6959 - acc: 0.5017 - val_loss: 0.6932 - val_acc: 0.4946
Epoch 5/20
15000/15000 [==============================] - 3s 224us/step - loss: 0.6952 - acc: 0.5039 - val_loss: 0.6931 - val_acc: 0.4957
Epoch 6/20
15000/15000 [==============================] - 3s 224us/step - loss: 0.6946 - acc: 0.5000 - val_loss: 0.6932 - val_acc: 0.4946
Epoch 7/20
15000/15000 [==============================] - 3s 223us/step - loss: 0.6935 - acc: 0.5036 - val_loss: 0.6931 - 